In [ ]:
def getSecret(secretName):
    linked_service = "ifmpmvault"
    akv_name = "ifm-vault"

    # Fetch the key from Azure Key Vault
    secretValue = mssparkutils.credentials.getSecret(
        linkedService=linked_service,
        akvName=akv_name, 
        secret=secretName)
    return secretValue

In [ ]:
#Help link to learn about AD authentication for API calls 
#https://learn.microsoft.com/en-us/rest/api/servicebus/get-azure-active-directory-token

In [ ]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import joblib

from azureml.core.model import Model

subscription_id = getSecret('subscription-id')
resource_group = getSecret('resource-group')
workspace_name = getSecret('workspace-name')

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# pull all metrics of best run
experiment_name = 'PredictiveMaintenanceExperiment'

for automl_run in ws.experiments[experiment_name].get_runs():
    best_run, fitted_model = automl_run.get_output()  # We are taking the first run. You can update this if you like to take a different run
    break

In [ ]:
feat_data = spark.sql('''select * from machine_data_features''')

split_date = "2022-12-21"
training = feat_data.filter(feat_data.dt_truncated < split_date)
testing = feat_data.filter(feat_data.dt_truncated >= split_date)

X_test = testing.toPandas()
drop_cols =['msdyn_customerassetid','dt_truncated','msdyn_name', 'msdyn_productname', 'modifiedon','component', 'failure','label_e']
# Remove the extra names if that are in the input_features list
input_features = [x for x in feat_data.columns if x not in set(drop_cols)]
pd_predictions = pd.DataFrame(fitted_model.predict(X_test[input_features]), columns=['prediction'])

X_test['prediction'] = pd_predictions['prediction'].tolist()

df_prediction = spark.createDataFrame(X_test)
df_prediction.write.mode("overwrite").saveAsTable("machine_failure_prediction_score")

In [ ]:
pip install azure-identity azure-servicebus

In [ ]:
import json
import requests
from pyspark.sql.functions import * 
import uuid
from datetime import datetime, timedelta

sql_stmt = '''select distinct p.msdyn_customerassetid,msdyn_name,prediction,score from
(
    (select distinct msdyn_customerassetid,prediction, 0.95 as score from  machine_failure_prediction_score where prediction != 0 limit 2) as p 
    inner join dataverse_d365env_generated_data.msdyn_customerasset as m on p.msdyn_customerassetid = m.msdyn_customerassetid
)''' #adjust this code to get predict_proba for score later
df_predictions = spark.sql(sql_stmt)

In [ ]:
import json
from azure.identity import DefaultAzureCredential,ManagedIdentityCredential
from azure.servicebus import ServiceBusClient, ServiceBusMessage


servicebus_queue_name = "sbmessage"

# Create a ServiceBusClient using the credential and namespace
servicebus_client = ServiceBusClient.from_connection_string(conn_str=getSecret('service-bus-connection-string'), logging_enable=True)

# Create a ServiceBusSender using the client and queue name
servicebus_sender = servicebus_client.get_queue_sender(servicebus_queue_name)

df = df_predictions.collect()

servicedate = str((datetime.now() + timedelta(days=0, hours=36)))

for row in df:
    pm_data = {
        "deviceid": "SimulatedThermostat-XSOL-WF3", #row.msdyn_name, 
        "eventtoken": str(uuid.uuid4()),
        "reading": str(row.score),
        "readingtype": "PredictiveMaintenance",
        "ruleoutput": "AlarmPredMaint",
        "time": str(datetime.now()),
        "servicebydate": servicedate,
        "score": str(row.score)
    }

    #response = requests.post(queueUrl, headers=headers, json=pm_data)
    pm_data_string = json.dumps(pm_data)
    message = ServiceBusMessage(pm_data_string)

    message = ServiceBusMessage(
       pm_data_string,
       content_type='application/json'
   )

    with servicebus_sender:
        response = servicebus_sender.send_messages(message)
        print(response)
        break